# IO

## 1. Paths

In [ ]:
const path = require('path');

### 1.1. Normalize

In [ ]:
{
    const dir = 'a/b/c/../d';
    
    const normalizedDir = path.normalize(dir);
    console.log(`* when path is "${dir}", then path.normalize(dir) is: "${normalizedDir}"`);
}

### 1.2. Join

In [ ]:
{
    const dir1 = 'a/b/c';
    const dir2 = '../x/y';
    const fn = 'foo.txt';
    
    const joinedPath = path.join(dir1, dir2, fn);
    console.log(`* when "dir1=${dir1}", "dir2=${dir2}", "fn=${fn}" then "path.join(dir1, dir2, fn)" is: "${joinedPath}"`);
}

### 1.3. Absolute path

In [ ]:
{
    const dir = '../a/b/c';
    
    const absPath = path.resolve(dir);
    console.log(`* when path is "${dir}", then path.resolve(dir) is: "${absPath}"`);
}

### 1.4. Relative path

In [ ]:
{
    const dir1 = 'a/b/c/d/';
    const dir2 = 'a/b/d/';
    
    const re = path.relative(dir1, dir2);
    console.log(`* relative path of "${dir1}" and "${dir2}" is: "${re}"`);
}

### 1.5. Name of directory

In [ ]:
{
    const fullpath = 'a/b/c/d.txt';
    
    const dirname = path.dirname(fullpath);
    console.log(`* dirname of "${fullpath}" is: "${dirname}"`);
}

### 1.6. File name of path

In [ ]:
{
    const fullpath = 'a/b/c/d.txt';
    
    const basename = path.basename(fullpath);
    console.log(`* basename of "${fullpath}" is: "${basename}"`);
}

### 1.7. File extenstion name

In [ ]:
{
    const fullpath = 'a/b/c/d.txt';
    
    const extname = path.extname(fullpath);
    console.log(`* extname of "${fullpath}" is: "${extname}"`);
}

## 2. File operator

In [ ]:
const fs = require('fs');

In [ ]:
async function rm(filename) {
    return new Promise((resolve, reject) => {
        fs.unlink(filename, err => {
            if (err) {
                reject(err);
            } else {
                resolve();
            }
        });
    });
}

In [ ]:
async function exist(filename) {
    return new Promise(resolve => {
        fs.exists(filename, exist => resolve(exist));
    });
}

### 2.1. Create empty file

In [ ]:
{
    async function touch(filename) {
        return new Promise((resolve, reject) => {
            fs.open(filename, 'w', (err, fd) => {
                if (err) {
                    reject(err);
                } else {
                    fs.close(fd, err => {
                        if (err) {
                            reject(err);
                        } else {
                            resolve();
                        }
                    });
                }
            });
        });
    }
    
    try {
        await touch('test.txt');
        
        const ok = await exist('test.txt');
        console.log(`* after file created, "test.txt" exist? ${ok}`)
    } finally{
        await rm('test.txt');
    }
}